Starting this project

In [26]:
!pip install geopandas
!pip install xlrd
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
# Climate data
climate = pd.read_csv("cmip6-x0.25_climatology_tasmax,rx1day.csv")

# ESG data
esgscore=pd.read_csv("esgdata1.csv")

# Economic & demographic data
gdp1 = pd.read_csv("gdp_pcap.csv")
pop = pd.read_csv("pop.csv")
sdi = pd.read_csv("sdi.csv")

# Map shapefile
world = gpd.read_file("ne_110m_admin_0_countries.shp")


In [28]:
print(climate.columns)

print(esgscore.columns)
print(gdp.columns)
print(pop.columns)
print(sdi.columns)


Index(['code', 'name', '1971-07'], dtype='object')
Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '2023 [YR2023]'],
      dtype='object')
Index(['country', '1800', '1801', '1802', '1803', '1804', '1805', '1806',
       '1807', '1808',
       ...
       '2091', '2092', '2093', '2094', '2095', '2096', '2097', '2098', '2099',
       '2100'],
      dtype='object', length=302)
Index(['country', '1800', '1801', '1802', '1803', '1804', '1805', '1806',
       '1807', '1808',
       ...
       '2091', '2092', '2093', '2094', '2095', '2096', '2097', '2098', '2099',
       '2100'],
      dtype='object', length=302)
Index(['country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')


In [51]:
## cleaning before merging
climate.rename(columns={'code':'ISO3'}, inplace=True)

# Keep only ISO3 and latest year (or average for period)
climate = climate[['ISO3','name' ,'1971-07']]  # Example: use only 1971-07 column initially
climate.rename(columns={'1971-07':'tasmax'}, inplace=True)

In [53]:
print(climate['tasmax'].describe())


baseline_K = 300

# Step 2: Convert anomaly to absolute temperature in Kelvin
climate['tasmax_K'] = climate['tasmax'] + baseline_K

# Step 3: Convert Kelvin to Celsius
climate['tasmax_C'] = climate['tasmax_K'] - 273.15

# Optional: Round for better readability
climate['tasmax_C'] = climate['tasmax_C'].round(2)

print(climate.head())

count    234.000000
mean       0.004274
std        0.097283
min       -0.220000
25%       -0.070000
50%       -0.010000
75%        0.070000
max        0.250000
Name: tasmax, dtype: float64
  ISO3             name  tasmax  tasmax_K  tasmax_C
0  ABW    Aruba (Neth.)   -0.16    299.84     26.69
1  AFG      Afghanistan   -0.10    299.90     26.75
2  AGO           Angola   -0.06    299.94     26.79
3  AIA  Anguilla (U.K.)   -0.09    299.91     26.76
4  ALA              ALA   -0.06    299.94     26.79


In [32]:
print(climate.columns)
climate.head(5)

Index(['ISO3', 'tasmax', 'tasmax_K', 'tasmax_C'], dtype='object')


,ISO3,tasmax,tasmax_K,tasmax_C
0,ABW,-0.16,299.84,26.69
1,AFG,-0.10,299.90,26.75
2,AGO,-0.06,299.94,26.79
3,AIA,-0.09,299.91,26.76
4,ALA,-0.06,299.94,26.79


In [33]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load ESG CSV
esg = pd.read_csv("esgdata1.csv")  # adjust filename

# Select relevant series
series_to_use = [
    'Adjusted savings: natural resources depletion (% of GNI)',
    'Adjusted savings: net forest depletion (% of GNI)',
    'CO2 emissions (metric tons per capita)',
    'Renewable energy consumption (% of total final energy consumption)',
    'Access to electricity (% of population)',
    'Access to clean fuels and technologies for cooking (% of population)',
    'Control of Corruption: Estimate',
    'Government Effectiveness: Estimate',
    'Rule of Law: Estimate',
    'Regulatory Quality: Estimate'
]

# Filter relevant indicators
esg_filtered = esg[esg['Series Name'].isin(series_to_use)]

# Pivot table: countries as rows, indicators as columns
esg_pivot = esg_filtered.pivot(index='Country Name', columns='Series Name', values='2023 [YR2023]').reset_index()

# Replace '..' or other non-numeric values with NaN
esg_pivot.replace('..', np.nan, inplace=True)

# Convert all columns to numeric (ignore Country Name)
for col in series_to_use:
    esg_pivot[col] = pd.to_numeric(esg_pivot[col], errors='coerce')

# Fill missing values (optional: with mean or median)
esg_pivot[series_to_use] = esg_pivot[series_to_use].fillna(esg_pivot[series_to_use].mean())

# Normalize each indicator
scaler = MinMaxScaler()
esg_normalized = esg_pivot.copy()
esg_normalized[series_to_use] = scaler.fit_transform(esg_pivot[series_to_use])

# Compute ESG proxy score
esg_normalized['ESG_Proxy'] = esg_normalized[series_to_use].mean(axis=1)

print(esg_normalized.head())


Series Name Country Name  \
0            Afghanistan   
1                Albania   
2                Algeria   
3                Andorra   
4                 Angola   

Series Name  Access to clean fuels and technologies for cooking (% of population)  \
0                                                          NaN                      
1                                                          NaN                      
2                                                          NaN                      
3                                                          NaN                      
4                                                          NaN                      

Series Name  Access to electricity (% of population)  \
0                                           0.844609   
1                                           1.000000   
2                                           1.000000   
3                                           1.000000   
4                                       

C:\Users\VIR MACHRA\AppData\Local\Temp\ipykernel_23696\288117901.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  esg_pivot.replace('..', np.nan, inplace=True)
c:\Python310\lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Python310\lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [34]:
esg_normalized

Series Name,Country Name,Access to clean fuels and technologies for cooking (% of population),Access to electricity (% of population),Adjusted savings: natural resources depletion (% of GNI),Adjusted savings: net forest depletion (% of GNI),CO2 emissions (metric tons per capita),Control of Corruption: Estimate,Government Effectiveness: Estimate,Regulatory Quality: Estimate,Renewable energy consumption (% of total final energy consumption),Rule of Law: Estimate,ESG_Proxy
0,Afghanistan,NaN,0.844609,NaN,NaN,NaN,0.187459,0.073380,0.238535,NaN,0.134567,0.295710
1,Albania,NaN,1.000000,NaN,NaN,NaN,0.376780,0.555122,0.544905,NaN,0.490041,0.593370
2,Algeria,NaN,1.000000,NaN,NaN,NaN,0.317619,0.356862,0.306340,NaN,0.366983,0.469561
3,Andorra,NaN,1.000000,NaN,NaN,NaN,0.741188,0.818737,0.795940,NaN,0.882713,0.847716
4,Angola,NaN,0.483087,NaN,NaN,NaN,0.312876,0.283935,0.346106,NaN,0.266314,0.338463
...,...,...,...,...,...,...,...,...,...,...,...,...
234,Viet Nam,NaN,0.997886,NaN,NaN,NaN,0.357543,0.528345,0.426785,NaN,0.508855,0.563883
235,World,NaN,0.911207,NaN,NaN,NaN,0.437671,0.485073,0.490291,NaN,0.511816,0.567211
236,"Yemen, Rep.",NaN,0.826638,NaN,NaN,NaN,0.073781,0.011295,0.115716,NaN,0.089262,0.223338
237,Zambia,NaN,0.483087,NaN,NaN,NaN,0.343300,0.359266,0.402186,NaN,0.402694,0.398106


In [35]:
# Drop rows with too many NaNs
esg_clean = esg_normalized.dropna(subset=['ESG_Proxy'])

# Check for duplicates or blanks
esg_clean = esg_clean.drop_duplicates(subset=['Country Name'])
esg_clean = esg_clean[~esg_clean['Country Name'].str.contains('World|Income|Region', na=False)]

print(esg_clean.shape)
print(esg_clean.head())


(237, 12)
Series Name Country Name  \
0            Afghanistan   
1                Albania   
2                Algeria   
3                Andorra   
4                 Angola   

Series Name  Access to clean fuels and technologies for cooking (% of population)  \
0                                                          NaN                      
1                                                          NaN                      
2                                                          NaN                      
3                                                          NaN                      
4                                                          NaN                      

Series Name  Access to electricity (% of population)  \
0                                           0.844609   
1                                           1.000000   
2                                           1.000000   
3                                           1.000000   
4                             

In [36]:
# Environmental
env_features = [
    'Adjusted savings: natural resources depletion (% of GNI)',
    'Adjusted savings: net forest depletion (% of GNI)',
    'CO2 emissions (metric tons per capita)',
    'Renewable energy consumption (% of total final energy consumption)',
    'Access to electricity (% of population)',
    'Access to clean fuels and technologies for cooking (% of population)',
]

# Social
social_features = [
    'Access to electricity (% of population)',
    'Access to clean fuels and technologies for cooking (% of population)',
]

# Governance
governance_features = [
    'Control of Corruption: Estimate',
    'Government Effectiveness: Estimate',
    'Rule of Law: Estimate',
    'Regulatory Quality: Estimate'
]

# Compute sub-scores
esg_clean['E_Score'] = esg_clean[env_features].mean(axis=1)
esg_clean['S_Score'] = esg_clean[social_features].mean(axis=1)
esg_clean['G_Score'] = esg_clean[governance_features].mean(axis=1)

# Recalculate overall ESG score
esg_clean['ESG_Final'] = esg_clean[['E_Score', 'S_Score', 'G_Score']].mean(axis=1)


In [37]:
scaler = MinMaxScaler()
esg_clean[['E_Score', 'S_Score', 'G_Score', 'ESG_Final']] = scaler.fit_transform(
    esg_clean[['E_Score', 'S_Score', 'G_Score', 'ESG_Final']]
)


In [38]:
# Sort countries by ESG_Final score
esg_clean = esg_clean.sort_values(by='ESG_Final', ascending=False)

# Save final dataset
esg_clean.to_csv("esg_final.csv", index=False)

print("✅ ESG Final Dataset Created! Top countries:")
print(esg_clean[['Country Name', 'E_Score', 'S_Score', 'G_Score', 'ESG_Final']].head(10))



✅ ESG Final Dataset Created! Top countries:
Series Name Country Name  E_Score  S_Score   G_Score  ESG_Final
190            Singapore      1.0      1.0  1.000000   1.000000
49               Denmark      1.0      1.0  0.986745   0.995734
71               Finland      1.0      1.0  0.961160   0.987500
211          Switzerland      1.0      1.0  0.956591   0.986029
129           Luxembourg      1.0      1.0  0.948915   0.983559
162               Norway      1.0      1.0  0.940301   0.980787
156          New Zealand      1.0      1.0  0.932669   0.978330
210               Sweden      1.0      1.0  0.916140   0.973011
155          Netherlands      1.0      1.0  0.914260   0.972406
9              Australia      1.0      1.0  0.907997   0.970390


Now that we have required data, ESG Final is cleaned
So, now let's clean all the other datasets then merge all to final 

In [39]:
print(sdi.columns)
print(pop.columns)
print(gdp.columns)
sdi.rename(columns={'country_name': 'Country', 'year': 'Year'}, inplace=True)
pop.rename(columns={'country_name': 'Country', 'year': 'Year'}, inplace=True)
gdp.rename(columns={'country_name': 'Country', 'year': 'Year'}, inplace=True)



Index(['country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
Index(['country', '1800', '1801', '1802', '1803', '1804', '1805', '1806',
       '1807', '1808',
       ...
       '2091', '2092', '2093', '2094', '2095', '2096', '2097', '2098', '2099',
       '2100'],
      dtype='object', length=302)
Index(['country', '1800', '1801', '1802', '1803', '1804', '1805', '1806',
       '1807', '1808',
       ...
       '2091', '2092', '2093', '2094', '2095', '2096', '2097', '2098', '2099',
       '2100'],
      dtype='object', length=302)


In [40]:
# Merge GDP and Population first
gdp_pop = pd.merge(gdp, pop, on=['country'], how='outer')

# Now merge SDI
merged_df = pd.merge(gdp_pop, sdi, on=['country'], how='outer')


In [41]:
merged_df.to_csv("final_econ_data.csv", index=False)


In [44]:
## testing this for country mismatch
final_econ=pd.read_csv('final_econ_data.csv')
final_esg=pd.read_csv('esg_final.csv')
from rapidfuzz import process

def match_country(name, choices):
    match = process.extractOne(name, choices)
    if match:
        return match[0]  # the matched string
    else:
        return None

# Get all unique countries in final_econ
econ_countries = final_econ['country'].unique()

# Map each ESG country to the closest match in econ
final_esg['matched_country'] = final_esg['Country Name'].apply(lambda x: match_country(x, econ_countries))

# Merge on matched_country
master_df = pd.merge(
    final_esg,
    final_econ,
    how='inner',
    left_on='matched_country',
    right_on='country'
)
master_df.to_csv('test_econ+esg.csv',index=False)



In [46]:
final_econ=pd.read_csv('final_econ_data.csv')
final_esg=pd.read_csv('esg_final.csv')
master_df=pd.merge(
    final_esg,
    final_econ,
    how='inner',
    left_on='Country Name',
    right_on='country'
)
master_df.to_csv('esg+econ.csv',index='False')


In [49]:
climate=pd.read_csv('cmip6-x0.25_climatology_tasmax,rx1day.csv')
climate.head(5)

,code,name,1971-07
0,ABW,Aruba (Neth.),-0.16
1,AFG,Afghanistan,-0.10
2,AGO,Angola,-0.06
3,AIA,Anguilla (U.K.),-0.09
4,ALA,ALA,-0.06


In [55]:


# Strip spaces from country names
climate['name'] = climate['name'].str.strip()

# Pick the column we downloaded (here it's '1971-07')
climate_selected = climate[['name', 'tasmax_C']].rename(columns={
    'name': 'Country Name',
    'tasmax_C': 'Temperature'
})
climate_selected.to_csv('final_climate.csv',index=False)
print(climate_selected.head())

      Country Name  Temperature
0    Aruba (Neth.)        26.69
1      Afghanistan        26.75
2           Angola        26.79
3  Anguilla (U.K.)        26.76
4              ALA        26.79


In [57]:
econ_esg=pd.read_csv('test_econ+esg.csv')
climate=pd.read_csv('final_climate.csv')

from rapidfuzz import process
econ_esg['Country Name'] = econ_esg['Country Name'].str.strip().str.lower()
climate['Country Name'] = climate['Country Name'].str.strip().str.lower()

def match_country(name, choices):
    match = process.extractOne(name, choices)
    if match:
        return match[0]  # the matched string
    else:
        return None

# Get all unique countries in econ_esg
econ_esg_countries = econ_esg['Country Name'].unique()

# Map each ESG country to the closest match in econ
climate['matched_country'] = climate['Country Name'].apply(lambda x: match_country(x, econ_esg_countries))

# Merge on matched_country
master_df = pd.merge(
    econ_esg,
    climate,
    how='inner',
    left_on='Country Name',
    right_on='matched_country'
)



master_df.to_csv('final_master_test.csv',index=False)
